# Walmart API overview
##### Search API used for look ups. We will use this first
##### Taxonomy API used for classifications.  We want to find a way to count classifications

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import urllib
import json
import sys
import pprint

# We are using 2 api Keys for redundancy and because
# there is a limit to the number of calls
api_key = "4tsbjxvbrwnpcjdsh277csqu"
api_alt = "anrtcd5dheh9758z5wgdy677"

# Base url for all API look ups
base_url = "http://api.walmartlabs.com/v1/" 

# Variables to specify the API
search = "search?"
paginated_products = "paginated/items?"

# Base url for search
search_url = base_url + search
paginated_url = base_url + paginated_products



# Search API

In [11]:
# Load search criteria for Walmart Search API.
# reference URL: https://developer.walmartlabs.com/docs/read/Search_API

format = "json"
categoryId = "3944"
query = "laptop"
numItems = 25 # max 25. 10 is default
start = 1
sort = "price"
order = "desc"
facet = "on" # boolean
min = str(0) # lower end of range for search within price range
max = str(200) # higher end of range price for search with price range
facet_range = (f"price:[{min}%20TO%20{max}]") # note the format [num1:num2] does not work. Use %20TO%20 for ":"



# NOTE: we are using 2 methods to create urls. Because the params argument does not format the output
# in the manner needed to get the API to work.  This is used only for price range lookups

# manual url. Used for price range lookup only
manual_url = (f"{search_url}apiKey={api_key}&format={format}&categoryId={categoryId}&query={query}&numItems={numItems}&facet={facet}&facet.range={facet_range}")

print(manual_url)

# API parameters for other lookups
params = {
    "apiKey": api_key,
    #"start": start, # note keep turned off.  We can't get this to work yet
    "format": "json", # json|xml
    "categoryId": categoryId,
    "query": query,
    "numItems": numItems,
    #"sort": sort, #allowed sort types are [relevance, price, title, bestseller, customerRating, new]
    #"order": order,  #allowed values are [asc, desc]
    #"facet":facet, # note keep turned off.  We can't get this to work yet
    #"facet.range":facet_range    # note keep turned off.  We can't get this to work yet
}

http://api.walmartlabs.com/v1/search?apiKey=4tsbjxvbrwnpcjdsh277csqu&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[0%20TO%20200]


In [12]:
# Loop by pricing parameters to pull out entire data set for a product category
# to save time we are going to hard code some numbers then go back later if we have time

# Number of items found in our search
# there are 20,747 total results for laptop in electronics
# step 0 there are 3089 results in the price range $0-$200
# step 1 there are 3015 results in the price range $201-$400
# step 2 there are 2053 results in the price range $401-$600
# step 3 there are 2995 results in the price range $601-$1000
# step 4 there are 2021 results in the price range $1001-$1400
# step 5 there are 2726 results in the price range $1401-$1800
# step 6 there are 1873 results in the price range $1801-$2200
# step 7 there are 1011 results in the price range $2201-$2600
# step 8 there are 863 results in the price range $2601-3000
# step 9 there are 1096 results in the price range above $3000

category_total = 20747 #hard coded from broad search
step0 = 200*25//3089
step1 = 200*25//3015
step2 = 200*25//2053
step3 = 400*25//2995
step4 = 400*25//2021
step5 = 400*25//2726
step6 = 400*25//1873
step7 = 400*25//1011
step8 = 400*25//863

# create a function to perform the price search

def price_data(lowRange, highRange):
    facet_range = (f"price:[{lowRange}%20TO%20{highRange}]")
    price_url = (f"{search_url}apiKey={api_alt}&format={format}&categoryId={categoryId}&query={query}&numItems={numItems}&facet={facet}&facet.range={facet_range}")
    walmart_data2 = requests.get(price_url)
    walmart_data2 = walmart_data2.json()
    
    try:       
        a = []
        if not os.path.isfile('Walmart_datasets2/walmart_more.json'):
            a.append(walmart_data2)
            with open('Walmart_datasets2/walmart_more.json', mode='w') as f:
                f.write(json.dumps(walmart_data2, indent=4))
        else:
            with open('Walmart_datasets2/walmart_more.json') as feedsjson:
                feeds = json.load(feedsjson)

            feeds.append(walmart_data2)
            with open('Walmart_datasets/walmart_more.json', mode='w') as f:
                f.write(json.dumps(feeds, indent=4))

    except:
        print ("Skipped line")
                


In [13]:
# Use for loop to go through prices in the steps defined

searchCount = 0
itemCount = 0
low = 0
high = 0

for itemCount in range (0,21000):
    itemCount = (searchCount+1)*25
    if high < 200:
        high = low + step0
        price_data(low,high)
        low = high
        searchCount += 1
        print(f"search / item count = {searchCount} / {itemCount}")
    elif high < 400:
        high = low + step1
        price_data(low,high)
        low = high
        searchCount += 1
        print(f"search / item count = {searchCount} / {itemCount}")
    elif high < 600:
        high = low + step2
        price_data(low,high)
        low = high
        searchCount += 1
        print(f"search / item count = {searchCount} / {itemCount}")
    elif high < 1000:
        high = low + step3
        price_data(low,high)
        low = high
        searchCount += 1
        print(f"search / item count = {searchCount} / {itemCount}")            
    elif high < 1400:
        high = low + step4
        price_data(low,high)
        low = high
        searchCount += 1
        print(f"search / item count = {searchCount} / {itemCount}")    
    elif high < 1800:
        high = low + step5
        price_data(low,high)
        low = high
        searchCount += 1
        print(f"search / item count = {searchCount} / {itemCount}")
    elif high < 2200:
        high = low + step6
        price_data(low,high)
        low = high
        searchCount += 1
        print(f"search / item count = {searchCount} / {itemCount}")        
    elif high < 2600:
        high = low + step7
        price_data(low,high)
        low = high
        searchCount += 1
        print(f"search / item count = {searchCount} / {itemCount}")
    elif high < 3000:
        high = low + step8
        price_data(low,high)
        low = high
        searchCount += 1
        print(f"search / item count = {searchCount} / {itemCount}")
    elif high < 8000:
        high = low + step9
        price_data(low,high)
        low = high
        searchCount += 1
        print(f"search / item count = {searchCount} / {itemCount}")
    else:
        print ('finished')
        print (f'item count = {itemCount}')
        print (f'final search count = {searchCount}')
        print (f'final high price is ${high}')
        break
      



search / item count = 1 / 25
Skipped line
search / item count = 2 / 50
Skipped line
search / item count = 3 / 75
Skipped line
search / item count = 4 / 100
Skipped line
search / item count = 5 / 125
Skipped line
search / item count = 6 / 150
Skipped line
search / item count = 7 / 175
Skipped line
search / item count = 8 / 200
Skipped line
search / item count = 9 / 225
Skipped line
search / item count = 10 / 250
Skipped line
search / item count = 11 / 275
Skipped line
search / item count = 12 / 300
Skipped line
search / item count = 13 / 325
Skipped line
search / item count = 14 / 350
Skipped line
search / item count = 15 / 375
Skipped line
search / item count = 16 / 400
Skipped line
search / item count = 17 / 425
Skipped line
search / item count = 18 / 450
Skipped line
search / item count = 19 / 475
Skipped line
search / item count = 20 / 500
Skipped line
search / item count = 21 / 525
Skipped line
search / item count = 22 / 550
Skipped line
search / item count = 23 / 575
Skipped line


Skipped line
search / item count = 183 / 4575
Skipped line
search / item count = 184 / 4600
Skipped line
search / item count = 185 / 4625
Skipped line
search / item count = 186 / 4650
Skipped line
search / item count = 187 / 4675
Skipped line
search / item count = 188 / 4700
Skipped line
search / item count = 189 / 4725
Skipped line
search / item count = 190 / 4750
Skipped line
search / item count = 191 / 4775
Skipped line
search / item count = 192 / 4800
Skipped line
search / item count = 193 / 4825
Skipped line
search / item count = 194 / 4850
Skipped line
search / item count = 195 / 4875
Skipped line
search / item count = 196 / 4900
Skipped line
search / item count = 197 / 4925
Skipped line
search / item count = 198 / 4950
Skipped line
search / item count = 199 / 4975
Skipped line
search / item count = 200 / 5000
Skipped line
search / item count = 201 / 5025
Skipped line
search / item count = 202 / 5050
Skipped line
search / item count = 203 / 5075
Skipped line
search / item count =

JSONDecodeError: Expecting value: line 3 column 1 (char 2)

In [8]:
from pprint import pprint
import os
filepath = os.path.join('Walmart_datasets2/walmart_more.json')
with open(filepath) as jsonfile:
    json_data = json.load(jsonfile)
    print (len(json_data))
    for i in range(len(json_data)):
        try:
            print (json_data['itemId'])
        except:
            print(f"skipped entry {i}")
    


FileNotFoundError: [Errno 2] No such file or directory: 'Walmart_datasets/walmart_more.json'